In [1]:
!pip3 install pandas


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/mnt/berry/home/linuxbrew/.linuxbrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
# from matplotlib import style
import random

# style.use("dark_background")
seed_val = 994
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device=torch.device("cuda:5")

In [3]:
# %history
train_df=pd.read_csv("./MELD/data/MELD/train_sent_emo.csv")
val_df=pd.read_csv("./MELD/data/MELD/dev_sent_emo.csv")
test_df=pd.read_csv("./MELD/data/MELD/test_sent_emo.csv")


train_df.head()




,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my company’s tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You must’ve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So let’s talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [4]:


print(train_df.Emotion.unique())
print(len(train_df))


['neutral' 'surprise' 'fear' 'sadness' 'joy' 'disgust' 'anger']
9989


In [5]:

mapping ={'neutral':0, 'surprise':1, 'fear':2, 'sadness':3, 'joy':4, 'disgust':5, 'anger':6}



train_df=train_df.replace({"Emotion":mapping})# mapping labels to integer
val_df=val_df.replace({"Emotion":mapping})
test_df=test_df.replace({"Emotion":mapping})
past_window=1

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# sid_obj = SentimentIntensityAnalyzer()

# mapping2={"positive":0,"neutral":1,"negative":2}
# train_sent=train_df.replace({"Sentiment":mapping2})
# train_sent=train_sent["Sentiment"].tolist()

# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score
# preds=[]
train_text=train_df["Utterance"].tolist()
val_text=val_df["Utterance"].tolist()
test_text=test_df["Utterance"].tolist()

# for i in (train_text):
#     sentiment_dict = sid_obj.polarity_scores(i)
# #     print(i)
#     if sentiment_dict['compound'] >= 0.7 :
#         preds.append(0)
#     elif sentiment_dict['compound'] <= - 0.7 :
#         preds.append(2)
#     else :
#         preds.append(1)
        
# print("acc=",accuracy_score(train_sent,preds))
# print(classification_report(train_sent, preds))

# print("f1 score", f1_score(train_sent,preds,average="weighted"))


train_labels=train_df["Emotion"]
val_labels=val_df["Emotion"]
test_labels=test_df["Emotion"]

# print(train_text,train_labels)

# len_train=[len(j) for j in train_text]
# len_test=[len(i) for i in test_text]

# pd.Series(len_train).hist()
# pd.Series(len_test).hist()

print(train_df.Emotion.value_counts())



0    4710
4    1743
1    1205
6    1109
3     683
5     271
2     268
Name: Emotion, dtype: int64


In [6]:
# print(train_text[6])




In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
bert = AutoModel.from_pretrained("roberta-large")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
tokens_train=tokenizer.batch_encode_plus(train_text,
                                         max_length=80*past_window,
                                         padding=True,
                                         truncation=True)
# tokens_train=tokenizer(train_text[3],
#                                          max_length=80*past_window,
#                                          padding=True,
#                                          truncation=True)
# print(tokens_train)
# print(tokenizer.decode(tokens_train["input_ids"]))

tokens_val=tokenizer.batch_encode_plus(val_text,
                                         max_length=80*past_window,
                                         padding=True,
                                         truncation=True)

tokens_test=tokenizer.batch_encode_plus(test_text,
                                         max_length=80*past_window,
                                         padding=True,
                                         truncation=True)

In [9]:
train_seq=torch.tensor(tokens_train["input_ids"])
train_mask=torch.tensor(tokens_train["attention_mask"])
train_label=torch.tensor(train_labels.tolist())

val_seq=torch.tensor(tokens_val["input_ids"])
val_mask=torch.tensor(tokens_val["attention_mask"])
val_label=torch.tensor(val_labels.tolist())

test_seq=torch.tensor(tokens_test["input_ids"])
test_mask=torch.tensor(tokens_test["attention_mask"])
test_label=torch.tensor(test_labels.tolist())

In [10]:
from torch.utils.data import DataLoader,TensorDataset,RandomSampler, SequentialSampler
batch_size=8

train_data=TensorDataset(train_seq,train_mask,train_label)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

val_data=TensorDataset(val_seq,val_mask,val_label)
val_sampler=SequentialSampler(val_data)
val_dataloader=DataLoader(val_data,sampler=val_sampler,batch_size=batch_size)

test_data=TensorDataset(test_seq,test_mask,test_label)
test_sampler=SequentialSampler(test_data)
test_dataloader=DataLoader(test_data,sampler=test_sampler,batch_size=batch_size)

In [11]:
from tqdm.notebook import tqdm_notebook
# modules=[bert.encoder.layer[:1]]
# for module in modules:
#     for param in module.parameters():
#         param.requires_grad=False

# for name, param in list(bert.named_parameters())[:-79]: 
#     print('I will be frozen: {}'.format(name)) 
#     param.requires_grad = False


In [12]:
class BERT_model(nn.Module):
    def __init__(self,bert):
        super(BERT_model,self).__init__()
        
        self.bert=bert
        
        self.dropout=nn.Dropout(0.1)
        
        self.relu=nn.ReLU()
        
#         self.fc1=nn.Linear(768,512)
        
        self.fc2=nn.Linear(1024,7)
        
        self.softmax=nn.LogSoftmax(dim=1)
        
    def forward(self, sent_id ,mask):
        
        cls_hs=self.bert(sent_id,attention_mask=mask)[1]
        
#         x=self.fc1(cls_hs)
        
#         x=self.relu(x)
        
        x=self.dropout(cls_hs)
        
        x=self.fc2(x)

        
        x=self.softmax(x)
        
        return x

In [13]:
model=BERT_model(bert)

model=model.to(device)

In [14]:
# !pip install torch==1.5.0
from transformers import AdamW

optimizer=AdamW(model.parameters(),
                lr=1e-5)


In [16]:
from sklearn.utils.class_weight import compute_class_weight

# class_weights=compute_class_weight("balanced",np.unique(train_labels),train_labels)
# print(class_weights)

In [17]:
# weights=torch.tensor(class_weights,dtype=torch.float)

# weights=weights.to(device)

# cross_entropy=nn.NLLLoss(weight=weights)  #not using is better
cross_entropy=nn.NLLLoss()

epochs=5
from transformers import get_linear_schedule_with_warmup
# scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0.01*len(train_dataloader),num_training_steps=batch_size*epochs)

In [20]:
def train():
    model.train()
    
    total_loss,total_accuracy=0,0
    
    total_preds=[]
    
    for step,batch in tqdm_notebook(enumerate(train_dataloader),total=len(train_dataloader)):
        
            
        batch=[r.to(device) for r in batch]
        
        sent_id,mask,labels=batch
        print(sent_id.requires_grad)

        optimizer.zero_grad()
        
        preds=model(sent_id,mask)
        
        
        loss=cross_entropy(preds,labels)
        
        
        total_loss=total_loss+loss.item()
        
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        
        
        optimizer.step()
#         scheduler.step()
        
        
        preds=preds.detach().cpu().numpy()
        total_preds.append(preds)
        
    avg_loss=total_loss/len(train_dataloader)
    
    total_preds=np.concatenate(total_preds,axis=0)
    
    
    return avg_loss,total_preds

ERROR! Session/line number was not unique in database. History logging moved to new session 233


In [21]:
def evaluate():
    print("\n Evaluating...")
     
    model.eval()
    
    total_loss,total_acc=0,0
    
    total_preds=[]
    
    for step,batch in tqdm_notebook(enumerate(val_dataloader),total=len(val_dataloader)):        
            
        batch=[r.to(device) for r in batch]
        
        sent_id,mask,labels=batch
        
        with torch.no_grad():
            
            preds=model(sent_id,mask)
            
            loss=cross_entropy(preds,labels)
            
            total_loss=total_loss + loss.item()
            
            preds=preds.detach().cpu().numpy()
            
            total_preds.append(preds)
            
    avg_loss=total_loss/len(val_dataloader)
    
    total_preds= np.concatenate(total_preds,axis=0)
    
    return avg_loss,total_preds

        
        

In [22]:
from sklearn.metrics import f1_score
best_val_loss=float("inf")

train_losses=[]
val_losses=[]

for epoch in range(epochs):
    print("\n For epoch{:}/{:}".format(epoch+1,epochs))
    
    train_loss,_=train()
    
    val_loss,val_preds_a=evaluate()

  
    print("shape",val_preds_a.shape)
    val_preds_a=np.argmax(val_preds_a, axis = 1)
#     torch.save(model.state_dict(),"/mnt/elm/prakhar/research/bert_no_context/no_context_roberta{:}.pt".format(epoch+1))
#     if val_loss<best_val_loss:
#         best_val_loss=val_loss
#         torch.save(model.state_dict(),"saved_weights-large-lossweight-40tok.pt")
        
#     print("\n Training loss=",train_loss)
    print("\n Validation loss=",val_loss)
    print("Report for val")
#     print((val_labels))
    print(classification_report(val_labels,val_preds_a))
    print("f1 score", f1_score(val_labels,val_preds_a,average="weighted"))


 For epoch1/5


  0%|          | 0/1249 [00:00<?, ?it/s]

False
False
False
False
False
False
False
False
False


KeyboardInterrupt: 

In [18]:
path = '/mnt/elm/prakhar/research/bert_no_context/no_context_roberta2.pt' #epoch1 best
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [19]:
torch.cuda.empty_cache()
total_preds=[]
with torch.no_grad():
  for step,batch in tqdm_notebook(enumerate(test_dataloader),total=len(test_dataloader)):        
            
        batch=[r.to(device) for r in batch]
        
        sent_id,mask,labels=batch
                
        preds=model(sent_id,mask)


        preds=preds.detach().cpu().numpy()

        total_preds.append(preds)
        
  total_preds= np.concatenate(total_preds,axis=0)

  print(total_preds)


  0%|          | 0/327 [00:00<?, ?it/s]

[[-0.4861045  -1.2867689  -4.3644743  ... -3.834118   -4.721577
  -3.1469903 ]
 [-0.4963631  -1.9692225  -3.0028994  ... -2.5135484  -4.8845344
  -2.6146753 ]
 [-0.13683453 -3.2623746  -3.7213635  ... -3.787551   -5.381347
  -3.7771282 ]
 ...
 [-1.9383972  -2.3093572  -4.054474   ... -0.8337363  -4.90138
  -1.2849299 ]
 [-2.5392857  -0.6000536  -2.3237314  ... -3.0497944  -3.226729
  -2.1581602 ]
 [-0.29470897 -4.7901554  -4.447216   ... -1.8653059  -4.292589
  -3.2417247 ]]


In [20]:
print(total_preds[0])

[-0.4861045 -1.2867689 -4.3644743 -3.7899516 -3.834118  -4.721577
 -3.1469903]


In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
total_preds = np.argmax(total_preds, axis = 1)
print(mapping)
print("acc=",accuracy_score(test_labels,total_preds))
print(classification_report(test_labels, total_preds))
print("f1 score", f1_score(test_labels,total_preds,average="weighted"))

{'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}
acc= 0.6517241379310345
              precision    recall  f1-score   support

           0       0.76      0.83      0.80      1256
           1       0.50      0.62      0.56       281
           2       0.29      0.26      0.27        50
           3       0.47      0.29      0.36       208
           4       0.56      0.65      0.60       402
           5       0.50      0.10      0.17        68
           6       0.58      0.40      0.47       345

    accuracy                           0.65      2610
   macro avg       0.52      0.45      0.46      2610
weighted avg       0.64      0.65      0.64      2610

f1 score 0.6367542302689448


In [17]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name,param.grad)

bert.embeddings.word_embeddings.weight None
bert.embeddings.position_embeddings.weight None
bert.embeddings.token_type_embeddings.weight None
bert.embeddings.LayerNorm.weight None
bert.embeddings.LayerNorm.bias None
bert.encoder.layer.0.attention.self.query.weight None
bert.encoder.layer.0.attention.self.query.bias None
bert.encoder.layer.0.attention.self.key.weight None
bert.encoder.layer.0.attention.self.key.bias None
bert.encoder.layer.0.attention.self.value.weight None
bert.encoder.layer.0.attention.self.value.bias None
bert.encoder.layer.0.attention.output.dense.weight None
bert.encoder.layer.0.attention.output.dense.bias None
bert.encoder.layer.0.attention.output.LayerNorm.weight None
bert.encoder.layer.0.attention.output.LayerNorm.bias None
bert.encoder.layer.0.intermediate.dense.weight None
bert.encoder.layer.0.intermediate.dense.bias None
bert.encoder.layer.0.output.dense.weight None
bert.encoder.layer.0.output.dense.bias None
bert.encoder.layer.0.output.LayerNorm.weight None
